# Bayers多模式集合方案

## 1.多模式集合背景介绍
动力气候模式是开展季度预测的重要工具。目前，利用动力气候模式进行季度预测主要有两种方法(Wang et al，2009)：一种是“两步法”，即在积分大气环流模式(AGCM)之前先预测出海表温度在目标时段的演变，强调海表温度对大气的强迫；另一种方法是基于海-气耦合模式(CGCM)的“一步法”。虽然目前海-气耦合模式仍存在较大的系统误差，但由于其具有模拟季风-海洋相互作用的能力而被认为是最具潜力的季风降水季度预测工具(曾庆存等，1990；Wu et al，2005；Wang et al，2005，2009)。

动力季度预测存在不确定性，因此，其本质上是一种概率预测，预测信息应由概率密度函数(PDF)完整地体现(Palmer et al，2005；Gneiting，2008；Doblas-Reyes et al，2009；Lavers et al，2009)。动力季度预测的不确定性包括初值不确定性和模式不确定性(黄嘉佑，1993；Palmer et al，2004；Weigel et al，2009)。其中，初值不确定性源于观测缺失、测量误差、同化分析误差等。而模式不确定性主要是模式对真实气候系统的简化造成的，如对次网格物理过程的参数化、各种模式参数的取值误差、不完美的边界条件等。与短期天气预报不同，对于季度预测，以上两种不确定性都很重要，都必须考虑。

## 2.数据预处理

In [11]:
%run ../src/Bayers.py -h

usage: Bayers.py [-h] [--EOF EOF] [-v] pretreat

positional arguments:
  pretreat       数据预处理

optional arguments:
  -h, --help     show this help message and exit
  --EOF EOF      观测降水数据经标准化处理后，作经验正交函数(EOF)展开并得到相应的EOF空间场
  -v, --verbose  输出详细信息


### 2.1 降水资料
所用的观测降水资料是由中国气象局国家气候中心提供的中国160站降水数据。

### 2.2 数据预处理

(1)降水数据经标准化处理后，作经验正交函数(EOF)展开，并根据特征值误差范围截取前5个主模态(累计方差贡献43%)，得到相应的EOF空间场；

(2)将这5个经验正交函数空间场作极大方差正交旋转得到5个旋转经验正交函数空间场(每个旋转经验正交函数空间场具有单个高荷载区)；

(3)根据各个旋转经验正交函数空间场的荷载值，划分区域；
此后，再将1960—2005年各区的区域平均降水量作为观测区域夏季降水时间序列。而模拟的区域夏季降水时间序列是通过先将模式输出的降水数据双线性插值到中国东部120个降水测站上，然后基于观测降水的5个分区计算得来。

由于集合成员数总是有限的，进行概率密度函数预测往往需要假设目标变量服从某种经典的概率分布，其中最常用的是正态分布假设。根据Jarque-Bera正态分布检验(Bera et al，1980)，R2、R3、R4、R5的区域平均夏季降水量满足正态分布假设(α=0.05)。对于不满足正态分布假设的R1区域平均夏季降水，采用立方根幂转换方法使之正态化。

In [14]:
%run ../src/Bayers.py --pretreat ../tmp/data --EOF all

数据预处理操作需要先将数据汇总至指定目录，程序自动搜索目录下每个文件，对其进行标准化操作，做经验正交函数(EOF)展开。

## 3.方法

考虑到模式后报资料较短，采用留一法(Wilks，1995)来评估不同多模式集合方案的概率密度函数预测技巧，即将除目标年外的数据资料作为训练集，进而建立集合模型来预测目标年的降水概率密度函数。通过留一法评估的模型技巧属于交叉验证技巧。

** Bayes**

贝叶斯集合方案是Coelho等(2004)提出的。该方案包括：

(1)选择先验分布；
(2)估计似然函数；
(3)确定后验分布。

首先，选取常用的气候学预测的概率密度函数作为先验概率密度函数。其次，对于服从正态分布的目标变量，可简单假设其似然函数服从正态分布(Coelho et al，2004)。似然函数N(aOt+b，γVt)可通过建立集合平均与观测值 O 间的一元线性回归方程来估计；其中，a和b为基于训练集估计的回归系数，Vt=(S2X)t/m，γ为残差平方的加权平均。最后，基于贝叶斯定理，当先验分布和似然函数均服从正态分布时，后验分布也服从正态分布。对于目标年t，其后验分布为:
![](../tmp/6_1.png)
其均值和方差分别由
![](../tmp/6_2.png)
![](../tmp/6_3.png)
来估算。

## 4.结果分析

### 4.1 校准度和锐度
校准度和锐度是概率密度函数预测的两个期望性质。概率预测的目标就是保证概率密度函数预测在校准条件下能最大化其锐度(Raftery et al，2005；Gneiting et al，2005，2007)。校准度(又称可靠度)用于衡量观测值和预测值的统计一致性，与观测值和预测值都有关。锐度(又称分辨率)表示预测的概率密度函数图像的精细程度，只与预测值有关。根据Raftery等(2005)和Gneiting等(2005，2007)的建议，采用概率积分变换PIT(Probability integral transform)直方图及95%置信区间相对宽度(RW)来分别评估校准度和锐度。
#### 4.1.1 校准度
概率积分变换值是预测的概率分布函数在不同观测值上的取值。概率积分变换直方图与确认等级直方图类似，差别在于前者用于评估概率密度函数预测的校准度，后者用于评估概率类别预测的校准度。 概率积分变换直方图的相关理论和计算方法可参见文献(Gneiting et al，2007)。当样本无限大时，如果概率密度函数预测是经过校准的，那么概率积分变换值应服从均匀分布(即概率积分变换直方图中不同概率区间的相对频率应相等)。在现实世界中，样本数总是有限的，概率积分变换值是否服从均匀分布可用非参数χ2拟合优度检验来判别(Pearson，1900)。

集合方案Cali-EE、MLR、Bayes在不同概率区间上所预测的概率密度函数相对频率比集合方案EE更为接近(图 2)，因此，其概率密度函数预测具有更高的校准度。根据χ2拟合优度检验(表 1)，在所有区域，Cali-EE、MLR、Bayes的概率积分变换值满足均匀分布假设(α=0.05)，概率密度函数预测具有高校准度。相反地，EE在除江淮流域外的所有地区都未通过均匀分布检验，即其概率密度函数预测在除江淮流域以外的区域具有低校准度。以上结果表明Cali-EE、MLR、Bayes具有校准预测产品的能力； Cali-EE具有高校准度表明，通过校准单个集合成员的统计平均属性也能达到校准集合预测产品的目的。而MLR和Bayes具有高校准度与Gneiting等(2005)和Coelho等(2004)的实验结果一致。

点击查看原图
图 2 5个区域基于不同多模式集合方案的概率积分变换(PIT)直方图(4种多模式集合方案分别是：等权集合(EE)、对单个集合成员先订正再等权集合(Cali-EE)、基于多元线性回归的集合方案(MLR)、基于贝叶斯统计学的集合方案(Bayes))
Fig. 2 Probability integral transform(PIT)histograms for different regions(rows) and different ensemble schemes(columns)The four ensemble schemes are the equally-weighted ensemble(EE)，EE for calibrated model-simulations(Cali-EE)，the ensemble scheme based on multiple linear regression analysis(MLR)，and the Bayesian ensemble scheme(Bayes)

EE具有低校准度的原因可通过概率积分变换直方图(图 2)的形状获知。EE的概率积分变换直方图在华南明显向右偏斜，表明EE预测的降水量总体上偏少。EE的概率积分变换直方图在长江流域明显呈钟形，表明其概率密度函数预测为超离散。而EE的概率积分变换直方图在中国北方东部和中 国北方中部区明显向左偏斜，则表明EE预测的降水量又总体上偏多。
#### 4.1.2 锐度
对于服从正态分布的目标变量，其95%(α=0.05)置信区间宽度为2×z1-α/2σ，其中，z1-α/2为标准正态分布的1-α/2分位点，σ为其标准偏差。本研究所用的RW是集合预测与气候学预测的95%置信区间宽度的比值，即RW=，其中，σY，t及σclim，t分别表示第t年基于集合方案及气候学预测的概率密度函数产品的标准偏差。RW越大，预测的概率密度函数图形越平坦，反之亦然。当RW小于1时，表明该集合预测比气候学预测锐度更高。

概率密度函数预测的技巧是由校准度和锐度共同决定的。当锐度固定后，概率密度函数预测的技巧正比于其校准度。然而，预测技巧与锐度间并非单调关系，只有当预测的概率密度函数产品为绝对校准时，其预测技巧才一定正比于其锐度。
### 4.2 技巧评估
Gneiting等(2007)推荐使用连续等级概率评分的时间平均值(CRPS)来定量估计概率密度函数预测的技巧，即综合评估校准度和锐度。CRPS代表了预测和观测的累积分布函数间的差异(Hersbach，2000)，类似于概率类别预测的Brier评分。由于气候学预测是判断降水概率密度函数季度预测是否有技巧的基准，一个集合预测模型需优于气候学预测，才能认为其具有预测技巧，因此，采用集合预测与气候学预测的CRPS的比值(RCRPS)代替CRPS作技巧评分。当RCRPS小于1时，表明该概率密度函数预测优于气候学预测，具有预测技巧；且RCRPS越小，预测技巧越高。



## 5.参考文献
陈桂英,赵振国.1998.短期气候预测评估方法和业务初估.应用气象学报,9(2):178-185

陈红.2003.IAP跨季度一年度滚动式动力学气候预测及其实时预测试验[D].北京:中国科学院大气物理研究所,62pp

丁一汇,李清泉,李维京等.2004.中国业务动力季节预报进展.气象学报,62(5):598-612

冯锦明,符淙斌.2007.不同区域气候模式对中国地区温度和降水的长期模拟比较.大气科学,31(5):805-814

黄嘉佑,1993.统计动力分析与预报.北京:气象出版社, 243pp

黄荣辉,李崇银,王绍武.2003.我国旱涝重大气候灾害及其形成机理研究.北京:气象出版社,483pp

柯宗建.2007.季节气候预测中多模式集合若干问题的研究[D].北京:中国科学院大气物理研究所,173pp

郎咸梅,王会军,姜大膀.2004.应用九层全球大气格点模式进行的跨季度短期气候预测系统性试验.地球物理学报,47(1):19-24

李清泉,丁一汇,张培群.2004.一个全球海气耦合模式跨季度汛期预测能力的初步检验和评估.气象学报,62(6):740-751

李维京,张培群,李清泉等.2005.动力气候模式预测系统业务化及其应用.应用气象学报,16(3):1-11

林朝晖,李旭,赵彦等.1998.IAP PSSCA的改进及其对1998年全国汛期旱涝形势的预测.气候与环境研究,3(4):339-348

柳艳香,郭裕福.2005.应用耦合模式进行2003年度气候预测试验.气候与环境研究,10(2):257-264

秦正坤.2007.短期气候数值预测的误差订正和超级集合方法研究[D].南京:南京信息工程大学,140pp

王会军.1997.试论短期气候预测的不确定性.气候与环境研究,2(4):333-338

王会军,孙建奇,郎咸梅等.2008.几年来我国气候年际变异和短期气候预测研究的一些新成果.大气科学,32(4):806-814

王绍武.2001.现代气候学研究进展.北京:气象出版社,453pp

王小玲,屠其璞.2002.我国旬降水年变化特征的区域分布.南京气象学院学报,25(4):518-524

魏凤英.1999.现代气候统计诊断与预测技术.北京:气象出版社,128-134

卫捷,张庆云,陶诗言.2005.2004年夏季短期气候集成预测及检验.气候与环境研究,10(1):19-31

曾庆存,袁重光,王万秋等.1990.跨季度气候距平数值预测试验.大气科学,14(1):10-25

Alessandri, A, Borrelli A, Navarra A, et al, 2011: Evaluation of probabilistic quality and value of the ENESMBLES multi-model seasonal forecast com-parison with DEMETER. Mon Wea Rev, 139: 581-607

Bera A K, Jarque C M. 1980. Efficient tests for normality, homoscedasticity and serial independence of regression residuals. Econ Lett, 6: 255-259

Chen L J, Chen D L, Wang H J, et al. 2009. Regionalization of precipitation regimes in China. Atmos Oceanic Sci Lett, 2: 301-307

Coelho C A S, Pezzulli S, Balmaseda M, et al. 2004. Forecast calibration and combination: A simple bayesian approach for ENSO. J Climate, 17:1504-1516

Doblas-Reyes F J, Weisheimer A, Déqué M, et al. 2009. Addressing model uncertainty in seasonal and annual dynamical seasonal forecasts. Quart J Roy Meteor Soc, 135: 1538-1559

Gneiting T, Raftery A E, Westveld A H, et al. 2005. Calibrated probabilistic forecasting using ensemble model output statistics and minimum CRPS estimation. Mon Wea Rev, 133: 1098-1118

Gneiting T, Balabdaoui F, Raftery A E. 2007. Probabilistic forecasts, calibration and sharpness. J Roy Stat Soc Ser B, 69: 243-268

Gneiting T. 2008. Probabilistic forecasting. J Roy Stat Soc Ser A, 171: 319-321

Hersbach H. 2000. Decomposition of the continuous ranked probability score for ensemble prediction systems. Wea Forecasting, 15: 559-570

Kang I S, Yoo J H. 2006. Examination of multi-model ensemble seasonal prediction methods using a simple climate system. Clim Dyn, 26: 285-294

Ke Z J, Zhang P Q, Dong W J, et al. 2009. A new way to improve seasonal prediction by diagnosing and correcting the intermodel systematic errors. Mon Wea Rev, 137: 1898-1907

Kharin V V, Zwiers F W. 2002. Climate predictions with multimodel ensembles. J Climate, 15: 793-799

Krishnamurti T N, Kishtawal C M, LaRaw T E, et al. 1999. Improved weather and seasonal climate forecasts from multi-model superensemble. Science, 285: 1548-1550

Lau K M, Kim K M, Lee J Y. 2004. Interannual variability, global teleconnection and potential predictability associated with the Asian summer monsoon//Chang C P. East Asian Monsoon. World Scientific, 153-176

Lavers D, Luo L, Wood E F. 2009. A multiple model assessment of seasonal climate forecast skill for applications. Geophys Res Lett, 36: L23711, doi:10.1029/2009GL041365

Li F, Zeng Q C, Li C F. 2009. A Bayesian scheme for probabilistic multi-model ensemble prediction of summer rainfall over the Yangtze River Valley. Atmos Oceanic Sci Lett, 2: 314-319

Luo L, Wood E F, Pan M. 2007. Bayesian merging of multiple climate model forecasts for seasonal hydrological predictions. J Geophys Res, 112: D10102, doi:10.1029/2006JD007655
North G R, Bell T L, Cahalan R F, et al. 1982. Sampling errors in the estimation of empirical orthogonal functions. Mon Wea Rev, 110: 699-706

Palmer T N, Alessandri A, Andersen U, et al. 2004. DEMETER: Development of a European multi-model ensemble system for seasonal to interannual prediction. Bull Amer Meteor Soc, 85: 853-872

Palmer T N, Doblas-Reyes F, Hagedorn R, et al. 2005. Probabilistic prediction of climate using multi-model ensembles: From basics to applications. Phil Trans R Soc B, 360: 1991-1998

Pearson K. 1900. On the criterion that a given system of deviations from the probable in the case of correlated system of variables is such that it can reasonably be supposed to have arisen from random sampling. Philos Mag, 50: 157-175

Peng P, Kumar A, Van den Dool H, et al. 2002. An analysis of multimodel ensemble predictions for seasonal climate anomalies. J Geophys Res, 107: 4710, doi:10.1029/2002JD002712

Raftery A E, Gneiting T, Balabdaoui F, et al. 2005. Using Bayesian Model Averaging to calibrate forecast ensembles. Mon Wea Rev, 133: 1155-1174

Tippett M K, Barnston A G, Robertson A W. 2007. Estimation of seasonal precipitation tercile-based categorical probabilities from ensembles. J Climate, 20: 2210-2228

Von Storch H, Zwiers F W. 1999. Statistical analysis in climate research. Cambridge University Press, 455pp

Wang B, Ding Q H, Fu X H, et al. 2005. Fundamental challenge in simulation and prediction of summer monsoon rainfall. Geophys Res Lett, 32: L15711. doi:10.1029/2005GL022734

Wang B, Lee J, Kang I, et al. 2009. Advance and prospectus of seasonal prediction: Assessment of the APCC/CliPAS 14-model ensemble retrospective seasonal prediction (1980-2004). Clim Dyn, 33: 93-117

Weigel A P, Liniger M A, Appenzeller C. 2009. Seasonal ensemble forecasts: Are recalibrated single models better than multimodels? Mon Wea Rev, 137: 1460-1479

Weisheimer A, Doblas-Reyes F J, Palmer T N, et al. 2009. ENSEMBLES: A new multi-model ensemble for seasonal-to-annual predictions-Skill and progress beyond DEMETER in forecasting tropical 

Pacific SSTs. Geophys Res Lett, 36: L21711, doi:10.1029/2009GL040896
Wilks D S. 1995. Statistical methods in the atmospheric sciences: An introduction. Academic Press, 467pp

Wu R, Kirtman B. 2005. Roles of Indian and Pacific Ocean air-sea coupling in tropical atmospheric variability. Clim Dyn, 25:155-170